In [1]:
import itertools
import sys
from pathlib import Path
from collections import defaultdict

from generate_all_functions import BinaryFunction

In [2]:
inputs = 3
outputs = 3
basis = 'aig'
circuits_dir = Path('./aig_res')

In [3]:
grouped_functions = BinaryFunction.all_functions_grouped(inputs, outputs, basis)

In [4]:
len(grouped_functions.keys())

8886

In [5]:
# for key, values in grouped_functions.items():
#     print(f"{key}: {values}")

In [6]:
all_functions = set(BinaryFunction.all_functions(inputs, outputs))

In [7]:
len(all_functions)

2763520

In [8]:
from math import factorial

def c(n, k):
    assert 0 <= k <= n
    return factorial(n) // factorial(k) // factorial(n - k)

In [9]:
assert len(all_functions) == c(2 ** (2 ** inputs), outputs)

In [10]:
get_class = {value: key for key, values in grouped_functions.items() for value in values}

In [11]:
len(get_class.keys())

2828544

In [12]:
def check_availability(function, circuits_dir):
    assert circuits_dir.exists()
    filename = '_'.join(function.truth_tables) + '.txt'
    filepath = circuits_dir / filename
    return filepath.exists()

In [13]:
bad_set = dict()
for function in all_functions:
    normalized = get_class[function]
    if not check_availability(normalized, circuits_dir):
        bad_set[function] = normalized

In [15]:
len(bad_set)

462992

In [16]:
def get_function_size(function, circuits_dir: Path) -> int:
    assert circuits_dir.exists()
    filename = '_'.join(function.truth_tables) + '.txt'
    filepath = circuits_dir / filename
    if not filepath.exists():
        return -1  # Function not found
    with filepath.open('r') as file:
        lines = file.readlines()
        function_size = len(lines) - 2
        assert function_size >= 0
        return function_size

In [17]:
classes = defaultdict(set)
functions = defaultdict(int)

for i, function in enumerate(all_functions):
    class_representative = get_class[function]
    function_size = get_function_size(class_representative, circuits_dir)
    classes[function_size].add(class_representative)
    functions[function_size] += 1
    
    if i % 100000 == 0 and i > 0:
        print(f'============{i}============', file=sys.stderr)
        for s in sorted(classes.keys()):
            print(f'{s} -> {len(classes[s])}, {functions[s]}', file=sys.stderr)
        print(f'============{i}============', file=sys.stderr)

print("Final table:")
for s in sorted(classes.keys()):
    print(f'{s} -> {len(classes[s])}, {functions[s]}')

============100000============
-1 -> 1745, 16801
3 -> 49, 426
4 -> 231, 2690
5 -> 722, 8188
6 -> 1534, 17930
7 -> 2313, 27624
8 -> 2160, 26193
9 -> 16, 149
============100000============
============200000============
-1 -> 1803, 33595
3 -> 51, 831
4 -> 231, 5406
5 -> 725, 16199
6 -> 1539, 36170
7 -> 2317, 55310
8 -> 2163, 52178
9 -> 16, 312
============200000============
============300000============
-1 -> 1819, 50376
3 -> 51, 1217
4 -> 232, 8040
5 -> 725, 24129
6 -> 1540, 54337
7 -> 2318, 82970
8 -> 2163, 78447
9 -> 16, 485
============300000============
============400000============
-1 -> 1826, 67093
3 -> 51, 1649
4 -> 232, 10558
5 -> 726, 32122
6 -> 1540, 72148
7 -> 2318, 110846
8 -> 2163, 104920
9 -> 16, 665
============400000============
============500000============
-1 -> 1832, 83791
3 -> 51, 2070
4 -> 232, 13242
5 -> 726, 40311
6 -> 1540, 90057
7 -> 2318, 138288
8 -> 2163, 131395
9 -> 16, 847
============500000============
============600000============
-1 -> 1834, 100311
3 

Final table:
-1 -> 1840, 462992
3 -> 51, 11840
4 -> 232, 72264
5 -> 726, 223640
6 -> 1540, 498720
7 -> 2318, 762128
8 -> 2163, 727264
9 -> 16, 4672


In [18]:
expected_functions = c(2 ** (2 ** inputs), outputs)
sanity_check = sum(functions.values()) == expected_functions
print(f"Sanity check: {sanity_check}")
print(f"Total functions enumerated: {sum(functions.values())}")
print(f"{expected_functions} functions expected")

Sanity check: True
Total functions enumerated: 2763520
2763520 functions expected


In [28]:
print(f"classes found rate: {1 - len(classes[-1]) / sum(map(len, classes.values()))}")
print(f"functions found rate: {1 - functions[-1] / sum(functions.values())}")

classes found rate: 0.7929327031285167
functions found rate: 0.8324629458082445


functions found rate: 0.8324629458082445
